In [ ]:
import pandas as pd
import os
import requests
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.visualization import simple_norm
# from astropy.utils.data import conf

In [ ]:
l2 = "C/2004 D1, C/2005 R4, C/2005 T3, C/2006 L1, C/2006 Q1, C/2007 N3, C/2007 R1, C/2008 FK75, C/2009 F2, C/2009 S3, C/2009 U5, C/2010 G2, C/2012 A1, C/2016 R2, C/2023 H5"
df_init = pd.DataFrame({"Comet": l2.split(", ")})
df_init

In [ ]:
def load_decam_fits(href, row, name):
    hdul = fits.open(href, memmap=True)

    coord = SkyCoord(row['Object_RA']*u.deg, row['Object_Dec']*u.deg, frame='icrs')
    print(f"Processing {name} at RA={coord.ra.deg}, Dec={coord.dec.deg}")
    
    is_found = False
    for i, hdu in enumerate(hdul):
        print(f"Checking HDU {i}...", end=' ')
        if i == 0:
            continue
        w = WCS(hdu.header)
        ny, nx = hdu.data.shape
        x_pix, y_pix = w.world_to_pixel(coord)
        in_fov = (0 <= x_pix < nx) and (0 <= y_pix < ny)
        
        if in_fov:
            is_found = True
            print(f"Found in HDU {i} at pixel ({x_pix:.1f}, {y_pix:.1f})")
            size = u.Quantity((1, 1), u.arcmin)
            cutout = Cutout2D(hdu.data, (x_pix, y_pix), size, wcs=w)
            
            hdu_out = fits.PrimaryHDU(data=cutout.data, header=cutout.wcs.to_header())
            out_filename = f'./comets/fits/decam_cutout_{idx}_hdu{i}.fits'
            if os.path.exists('./comets/fits/') is False:
                os.makedirs('./comets/fits/')
                
            hdu_out.writeto(out_filename, overwrite=True)
            print(f'Wrote cutout to {out_filename}')    
            
            fig = plt.figure(figsize=(5, 5))
            ax = fig.add_subplot(1, 1, 1, projection=cutout.wcs)
            norm = simple_norm(cutout.data, 'sqrt', percent=99.)
            ax.imshow(cutout.data, origin='lower', cmap='gray', norm=norm,
                    )
            ax.scatter(cutout.wcs.world_to_pixel(coord)[0], cutout.wcs.world_to_pixel(coord)[1],
                    s=100, edgecolor='red', facecolor='none', marker='o', lw=1, alpha=0.7)
            ax.set_title(f'DEcam cutout HDU {i}')
            # WCSAxes niceties
            ax.set_xlabel("RA")
            ax.set_ylabel("Dec")
            ax.coords.grid(True, alpha=0.3, linestyle="--")
            plt.show()
        print()
        
    if not is_found:
        print("Comet not found in any HDU.")

In [ ]:
scan = os.scandir('data/cadc2/')
files = [entry.path for entry in scan if entry.is_file()]
files.sort()

# Set the cache directory to a specific path
# conf.cache_dir = './cache/astropy/'

for idx in range(0, len(files)):
    comet_name = "_".join(files[idx].split('/')[-1].split('.')[0].split('_')[:-1])
    print(f'Processing comet: {comet_name}')
    
    df_cadc = pd.read_csv(files[idx])
    decam_sum = (df_cadc['Telescope/Instrument'] == 'CTIO-4m/DECam').sum()

    if decam_sum == 0:
        print("DECam files did not found")
        continue
    else:
        print(f"Found {decam_sum} DECam files")
    
    df_jplh = pd.read_csv(f'data/jpl2/{comet_name}_jplhorizons.csv')
    df_merged = pd.merge(df_cadc, df_jplh, left_index=True, right_index=True, suffixes=('_cadc', '_jplh'))

    # Select observations with DECam
    df_decam = df_merged[df_merged['Telescope/Instrument'] == 'CTIO-4m/DECam']
    
    for i, row in df_decam.iterrows():
        print(f"  Processing row {i} with observation ID {row['Image']}")
        load_decam_fits(row['MetaData'], row, comet_name)
        break
    # break


In [ ]:
w = WCS(hdul[1].header)
ny, nx = hdul[1].data.shape
coord = SkyCoord(row['Object_RA']*u.deg, row['Object_Dec']*u.deg, frame='icrs')
x_pix, y_pix = w.world_to_pixel(coord)
print(f"Comet at pixel coordinates: x={x_pix}, y={y_pix}")
plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=w)
ax.scatter(x_pix, y_pix, s=100, edgecolor='red', facecolor='none', marker='o', lw=1, alpha=0.7)
norm = simple_norm(hdul[1].data, 'sqrt', percent=99.)
ax.imshow(hdul[1].data, cmap='gray', origin='lower', norm=norm)
ax.set_xlabel("RA")
ax.set_ylabel("Dec")
ax.coords.grid(True, alpha=0.3, linestyle="--")
plt.show()

In [ ]:
df_decam

In [ ]:
def cutout_decam(hdul, idx):

    coord = SkyCoord(df_decam.iloc[idx]['Object_RA']*u.deg, df_decam.iloc[idx]['Object_Dec']*u.deg, frame='icrs')
    for i, hdu in enumerate(hdul):
        if i == 0:
            continue
        w = WCS(hdu.header)
        ny, nx = hdu.data.shape
        x_pix, y_pix = w.world_to_pixel(coord)
        in_fov = (0 <= x_pix < nx) and (0 <= y_pix < ny)
        
        if in_fov:
            size = u.Quantity((1, 1), u.arcmin)
            cutout = Cutout2D(hdu.data, (x_pix, y_pix), size, wcs=w)
            
            hdu_out = fits.PrimaryHDU(data=cutout.data, header=cutout.wcs.to_header())
            out_filename = f'./comets/fits/decam_cutout_{idx}_hdu{i}.fits'
            if os.path.exists('./comets/fits/') is False:
                os.makedirs('./comets/fits/')
                
            hdu_out.writeto(out_filename, overwrite=True)
            print(f'Wrote cutout to {out_filename}')    
            
            fig = plt.figure(figsize=(5, 5))
            ax = fig.add_subplot(1, 1, 1, projection=cutout.wcs)
            norm = simple_norm(cutout.data, 'sqrt', percent=99.)
            ax.imshow(cutout.data, origin='lower', cmap='gray', norm=norm,
                    )
            ax.scatter(cutout.wcs.world_to_pixel(coord)[0], cutout.wcs.world_to_pixel(coord)[1],
                    s=100, edgecolor='red', facecolor='none', marker='o', lw=1, alpha=0.7)
            ax.set_title(f'DEcam cutout HDU {i}')
            # WCSAxes niceties
            ax.set_xlabel("RA")
            ax.set_ylabel("Dec")
            ax.coords.grid(True, alpha=0.3, linestyle="--")
            plt.show()
# hdul.close()